In [32]:
# Initialize notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True
SAVETABS = True

In [33]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [62]:
# plots Pathfinder PWL XP along with theoretical value

x  = np.array([-4,-3,-2,-1, 0, 1, 2, 3, 4])
y  = np.array([18,21,26,32,40,48,60,72,90])
#yf = np.array([10,15,20,30,40,60,80,120,160])*np.power(13/12, -2*x)
yf = 40*np.power(2, 0.5*x)*np.power(14/13, -2*x)

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
        ),
        yaxis=dict(
            title_text='XP',
            range=[0, 100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='PWL',
    hovertemplate=
        'level %{x}<br>'+
        'PWL %{y:.0f} XP' + 
        '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x,
    y=yf,
    mode='markers+lines',
    name='theory',
    hovertemplate=
        'level %{x}<br>'+
        'theory %{y:.0f} XP' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-theory-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-theory-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [65]:
# create PF2 monster table
def pf2_XP_standard(row):
    return 0.25*(1 + np.power(14/13, - 5))*row['Hit Points']*row['Strike Damage']*np.power(14/13, (row['Armor Class'] + row['Strike Attack Bonus'] - 20))

def pf2_XP_PWL(row):
    return 0.25*(1 + np.power(14/13, - 5))*row['Hit Points']*row['Strike Damage']*np.power(14/13, (row['Armor Class'] + row['Strike Attack Bonus'] - 2*row['Level'] - 20))

PF2_MONSTER_STATS = {
    'Level': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    'Hit Points': [7.5, 15.0, 20.0, 30.0, 45.0, 60.0, 75.0, 95.0, 115.0, 135.0, 155.0, 175.0, 195.0, 215.0, 235.0, 255.0, 275.0, 295.0, 315.0, 335.0, 355.0, 375.0, 400.0, 430.0, 460.0, 500.0],
    'Armor Class': [14, 15, 15, 17, 18, 20, 21, 23, 24, 26, 27, 29, 30, 32, 33, 35, 36, 38, 39, 41, 42, 44, 45, 47, 48, 50],
    'Strike Damage': [2.5, 4.5, 5.5, 8.5, 10.5, 12.0, 13.0, 15.0, 17.0, 18.0, 20.0, 22.0, 23.0, 25.5, 27.5, 28.5, 30.5, 31.5, 32.5, 33.5, 35.0, 37.0, 38.0, 40.0, 42.0, 44.0],
    'Strike Attack Bonus': [6, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31, 33, 34, 36, 37, 39, 40, 42],
}
dfMon = pd.DataFrame(PF2_MONSTER_STATS)

# save table
if SAVETABS:
    file_name = './table-pf-monster-moderate.html'
    tab_html = dfMon.to_html(index=False, float_format='{:,.1f}'.format, border=0)
    tab_html = tab_html.replace('<th>', '<th style="text-align: center;">')
    tab_html = tab_html.replace('<td>', '<td style="text-align: right;">')

    tab_soup = BeautifulSoup(tab_html, 'html.parser')
    tab_soup.table['class'] = ''

    div = tab_soup.new_tag('div')
    div['class'] = 'dataframe center'
    div['style'] = 'width:660px;'
    tab_soup.table.wrap(div)

    h3 = tab_soup.new_tag('h3')
    h3['id'] = 'tab:pf-monster-moderate'
    h3.string = 'Pathfinder 2 Monster Stats'
    tab_soup.div.insert(0, h3)

    with open(file_name, 'wb') as fout:
        fout.write(tab_soup.prettify('utf-8'))

dfMon['XP'] = dfMon.apply(lambda row: round(pf2_XP_standard(row), 0), axis=1)
dfMon['XP PWL'] = dfMon.apply(lambda row: round(pf2_XP_PWL(row), 0), axis=1)

In [74]:
# plots D&D XP vs CR along with an exponential fit.
dfD0 = pd.read_csv('../../assets/data/dmg-monster-xp.csv') # 'CR','XP'
dfD0 = dfD0[dfD0['CR'].between(1,30)]

x = dfD0['CR']
y = dfD0['XP']
ylog = np.log(dfD0['XP'])/np.log(2)

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP',
            type='log',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='DMG',
    hovertemplate=
        'CR %{x}<br>'+
        'DMG %{y:,.0f} XP' + 
        '<extra></extra>'
))

# add fit line
coefs = np.polyfit(x[0:-1], ylog[0:-1], 1)
print(coefs)
#name = '$\mathrm{fit} = 2^{' + '{:.3f} LV + {:.3f}'.format(coefs[0], coefs[1]) + '}$'
name = 'fit'
poly = np.poly1d(coefs)
"""fig.add_trace(go.Scatter(
    x=x,
    y=np.power(2, poly(x)),
    #y=np.power(2, 0.286*x)*200000/386,
    mode='lines',
    name=name,
    hovertemplate=
        'CR %{x}<br>'+
        'fit %{y:,.0f} XP' + 
        '<extra></extra>'
))"""

monLvl = dfMon[dfMon['Level'].between(1,24)]['Level'].to_numpy()
monXP = dfMon[dfMon['Level'].between(1,24)]['XP PWL'].to_numpy()
fig.add_trace(go.Scatter(
    x=monLvl, 
    #y=50*monXP/monXP[0],
    y=monXP,
    mode='markers+lines', 
    name='calculated XP',
    hovertemplate=
        '<b>PF 2e - standard</b><br>'+
        'Level %{x}<br>'+
        'XP %{y:,.0f}' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-dnd-xp-fit-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-dnd-xp-fit-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

[0.28828775 9.08856355]


In [86]:
# Plots monster XP relative to a monster two levels lower


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster level',
            range=[-0.5,25.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,6],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

monLvl = dfMon[dfMon['Level'].between(1,24)]['Level'].to_numpy()
monXP = dfMon[dfMon['Level'].between(1,24)]['XP PWL'].to_numpy()
fig.add_trace(go.Scatter(
    x=monLvl[2:], 
    y=monXP[2:]/monXP[0:-2],
    mode='markers+lines', 
    name='PF 2e PWL',
    hovertemplate=
        'Level %{x}<br>'+
        'XP ratio %{y:.2f}' + 
        '<extra></extra>'
))

dfD0 = pd.read_csv('../../assets/data/dmg-monster-xp.csv') # 'CR','XP'
dfD0 = dfD0[dfD0['CR'].between(1,24)]
monLvl = dfD0['CR'].to_numpy()
monXP = dfD0['XP'].to_numpy()
fig.add_trace(go.Scatter(
    x=monLvl[2:], 
    y=monXP[2:]/monXP[0:-2],
    mode='markers+lines', 
    name='D&D 5e',
    hovertemplate=
        'Level %{x}<br>'+
        'XP ratio %{y:.2f}' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

In [36]:
# Plots monster XP values calculated using XP equation along with the expected scaling exponential
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster level',
            range=[-0.5,25.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP',
            type='log',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

monLvl = dfMon[dfMon['Level'].between(1,24)]['Level'].to_numpy()
monXP = dfMon[dfMon['Level'].between(1,24)]['XP'].to_numpy()
fig.add_trace(go.Scatter(
    x=monLvl, 
    y=monXP/monXP[0],
    mode='markers+lines', 
    name='calculated XP',
    hovertemplate=
        '<b>PF 2e - standard</b><br>'+
        'Level %{x}<br>'+
        'XP %{y:,.0f}' + 
        '<extra></extra>'
))


coefs = np.polyfit(monLvl[4:], (np.log(monXP)/np.log(2))[4:], 1)
poly = np.poly1d(coefs)
fig.add_trace(go.Scatter(
    x=monLvl,
    y=6*np.power(2, (monLvl + 1)/2),
    mode='lines',
    name=r'$2^{L / 2}$',
    hovertemplate=
        'Level %{x}<br>'+
        'XP %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-vs-level-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-vs-level-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [77]:
# Plots monster XP relative to a monster two levels lower
monLvl = dfMon['Level'].to_numpy()
monXP = dfMon['XP'].to_numpy()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster level',
            range=[-0.5,25.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP / XP-2',
            range=[0,8],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=monLvl[2:], 
    y=monXP[2:]/monXP[0:-2],
    mode='markers+lines', 
    name='PF2 moderate',
    hovertemplate=
        'Level %{x}<br>'+
        'XP ratio %{y:.2f}' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-scaling-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-scaling-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [30]:
# plots the encounter multiplier for systems that assume 1 monster by default and 4 monsters by default

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def number_of_monsters(enc_xp, mon_xp):
    return np.floor(enc_xp/mon_xp)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='number of monsters',
            range=[0,21],
            tick0=0, dtick=2,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0.0,4.5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

#x = [1,2,3,7,11,15]
x = np.array(range(1,21))
y = [0.5*encounter_multiplier_DMG(4, n) for n in x]
fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='4 monsters',
    marker=dict(size=6),
    hovertemplate=
        '<b>PF 2e</b><br>'+
        '<b>Monsters:</b> %{x}<br>'+
        '<b>Multiplier:</b> %{y:.2f}'+ 
        '<extra></extra>'
))

#x = [1,2,3,7,11,15]
x = np.array(range(1,21))
y = [encounter_multiplier_DMG(4, n) for n in x]
fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='1 monster',
    marker=dict(size=6),
    hovertemplate=
        '<b>D&D 5e</b><br>'+
        '<b>Monsters:</b> %{x}<br>'+
        '<b>Multiplier:</b> %{y:.2f}'+ 
        '<extra></extra>'
))

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-encounter-multiplier-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-encounter-multiplier-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# show figure
fig.show(config=TFB_CONFIG)

In [63]:
# plots the encounter multiplier as well as the difference in critical hit chance

rel_level = [-4, -3, -2, -1, 0, 1, 2, 3, 4]
mon_xp = [10, 15, 20, 30, 40, 60, 80, 120, 160]
#enc_diff = ['Trivial', 'Low', 'Moderate', 'Severe', 'Extreme']
#enc_xp = [40, 60, 80, 120, 160]
enc_diffs = [
    dict(diff='Trivial',  xp=40,  color='blue'),
    dict(diff='Low',      xp=60,  color='orange'),
    dict(diff='Moderate', xp=80,  color='green'),
    dict(diff='Severe',   xp=120, color='red'),
    dict(diff='Extreme',  xp=160, color='purple'),
]

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def number_of_monsters(enc_xp, mon_xp):
    return np.floor(enc_xp/mon_xp)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP multiplier',
            range=[0,20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

for ed in enc_diffs:
    #n_mon = [number_of_monsters(ed['xp'], xp) for xp in mon_xp]
    #n_mon = np.array([n for n in n_mon if n > 0])
    n_mon = np.array([ed['xp']/xp for xp in mon_xp if xp < ed['xp']])
    fig.add_trace(go.Scatter(
        x=rel_level[0:len(n_mon)], 
        y=n_mon,
        mode='markers+lines', 
        marker=dict(size=6, color=ed['color']),
        name=ed['diff'],
        showlegend=False,
        legendgroup=ed['diff'],
        hovertemplate=
            'Level %{x}<br>'+
            'Monsters %{y:.0f}'+ 
            '<extra></extra>'
    ))
    """coefs = np.polyfit(rel_level[0:len(n_mon)], n_mon, 2)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=rel_level[0:len(n_mon)], 
        y=poly(rel_level[0:len(n_mon)]),
        mode='lines', 
        line=dict(color=ed['color'], dash='solid'),
        name=ed['diff'],
        legendgroup=ed['diff'],
        hoverinfo='skip',
    ))"""

# save large format figure
"""if SAVEFIGS:
    file_name = f'./fig-pf-xp-multiplier-components-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-multiplier-components-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))"""

# show figure
fig.show(config=TFB_CONFIG)

In [64]:
# plots chance to crit and be crit vs level

rel_level = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='monster relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='crit %',
            range=[0,50],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


fig.add_trace(go.Scatter(
    x=rel_level, 
    y=[max(5, y) for y in (15 - 5*rel_level)],
    mode='markers+lines', 
    marker=dict(size=6),
    name='player characters',
    legendgroup=ed['diff'],
    hovertemplate=
        'Level %{x}<br>'+
        'Crit Chance %{y:.0f}'+ 
        '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=rel_level, 
    y=[max(5, y) for y in (15 + 5*rel_level)],
    mode='markers+lines', 
    marker=dict(size=6),
    name='monsters',
    legendgroup=ed['diff'],
    hovertemplate=
        'Level %{x}<br>'+
        'Crit Chance %{y:.0f}'+ 
        '<extra></extra>'
))

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-crit-probability-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-crit-probability-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# show figure
fig.show(config=TFB_CONFIG)

In [52]:
# plots the encounter multiplier as well as the difference in critical hit chance

rel_level = [-4, -3, -2, -1, 0, 1, 2, 3, 4]
mon_xp = [10, 15, 20, 30, 40, 60, 80, 120, 160]
#enc_diff = ['Trivial', 'Low', 'Moderate', 'Severe', 'Extreme']
#enc_xp = [40, 60, 80, 120, 160]
enc_diffs = [
    dict(diff='Trivial',  xp=40,  color='blue'),
    dict(diff='Low',      xp=60,  color='orange'),
    dict(diff='Moderate', xp=80,  color='green'),
    dict(diff='Severe',   xp=120, color='red'),
    dict(diff='Extreme',  xp=160, color='purple'),
]

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def number_of_monsters(enc_xp, mon_xp):
    return np.floor(enc_xp/mon_xp)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP multiplier',
            range=[0.45,2.5],
            #tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=rel_level, 
    y=np.power((14/13), 2*np.array(rel_level)),
    mode='markers+lines', 
    name='crit modifier',
    marker=dict(size=6, color='black'),
    line=dict(color='black', dash='solid'),
    hovertemplate=
        'Level %{x}<br>'+
        '%{y:.2f}'+ 
        '<extra></extra>'
))

for ed in enc_diffs:
    n_mon = np.array([ed['xp']/xp for xp in mon_xp if xp < ed['xp']])
    em = [0.5*encounter_multiplier_DMG(4, n) for n in n_mon if n > 0]
    fig.add_trace(go.Scatter(
        x=rel_level[0:len(em)], 
        y=em,
        mode='markers', 
        marker=dict(size=6, color=ed['color']),
        name=ed['diff'],
        showlegend=False,
        legendgroup=ed['diff'],
        hovertemplate=
            'Level %{x}<br>'+
            '%{y:.2f}'+ 
            '<extra></extra>'
    ))
    coefs = np.polyfit(rel_level[0:len(em)], np.log(em), 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=rel_level[0:len(em)], 
        y=np.exp(poly(rel_level[0:len(em)])),
        mode='lines', 
        line=dict(color=ed['color'], dash='solid'),
        name=ed['diff'],
        legendgroup=ed['diff'],
        hoverinfo='skip',
    ))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-multiplier-components-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-multiplier-components-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [55]:
# plots the encounter multiplier as well as the difference in critical hit chance

rel_level = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])
mon_xp = np.array([10, 15, 20, 30, 40, 60, 80, 120, 160])
#enc_diff = ['Trivial', 'Low', 'Moderate', 'Severe', 'Extreme']
#enc_xp = [40, 60, 80, 120, 160]
enc_diffs = [
    dict(diff='Trivial',  xp=40,  color='blue'),
    dict(diff='Low',      xp=60,  color='orange'),
    dict(diff='Moderate', xp=80,  color='green'),
    dict(diff='Severe',   xp=120, color='red'),
    dict(diff='Extreme',  xp=160, color='purple'),
]

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def number_of_monsters(enc_xp, mon_xp):
    return np.floor(enc_xp/mon_xp)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP multiplier',
            #autorange=True,
            range=[0,1.4],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            #range=[0,220],
            #tick0=0, dtick=40,
            #minor=dict(tick0=0, dtick=20),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

for ed in enc_diffs:
    #n_mon = np.array([number_of_monsters(ed['xp'], xp) for xp in mon_xp])
    n_mon = np.array([ed['xp']/xp for xp in mon_xp])
    #em = [0.5*encounter_multiplier_DMG(4, number_of_monsters(ed['xp'], xp)) for xp in mon_xp]
    em = np.array([0.5*encounter_multiplier_DMG(4, n) for n in n_mon])
    crit = np.power((14/13), 2*rel_level)
    #xp = n_mon*mon_xp*em*crit
    xp = em*crit
    mask = n_mon >= 1
    x = rel_level[mask]
    y = xp[mask]
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        mode='markers', 
        marker=dict(size=6, color=ed['color']),
        name=ed['diff'],
        showlegend=False,
        legendgroup=ed['diff'],
        hovertemplate=
            'Level %{x}<br>'+
            '%{y:.2f}'+ 
            '<extra></extra>'
    ))
    coefs = np.polyfit(x, y, 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=x, 
        y=poly(x),
        mode='lines', 
        line=dict(color=ed['color'], dash='solid'),
        name=ed['diff'],
        legendgroup=ed['diff'],
        hoverinfo='skip',
    ))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-adjusted-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-adjusted-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [13]:
# table of average XP ratio value after accounting for group size and crit chance.

rel_level = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])
mon_xp = np.array([10, 15, 20, 30, 40, 60, 80, 120, 160])
#enc_diff = ['Trivial', 'Low', 'Moderate', 'Severe', 'Extreme']
#enc_xp = [40, 60, 80, 120, 160]
enc_diffs = [
    dict(diff='Trivial',  xp=40,  color='blue'),
    dict(diff='Low',      xp=60,  color='orange'),
    dict(diff='Moderate', xp=80,  color='green'),
    dict(diff='Severe',   xp=120, color='red'),
    dict(diff='Extreme',  xp=160, color='purple'),
]

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def number_of_monsters(enc_xp, mon_xp):
    return np.floor(enc_xp/mon_xp)

# create figure


for ed in enc_diffs:
    n_mon = np.array([number_of_monsters(ed['xp'], xp) for xp in mon_xp])
    #em = [0.5*encounter_multiplier_DMG(4, number_of_monsters(ed['xp'], xp)) for xp in mon_xp]
    em = np.array([0.5*encounter_multiplier_DMG(4, n) for n in n_mon])
    crit = np.power((14/13), 2*rel_level)
    mod = em*crit
    xp0 = n_mon*mon_xp
    xp1 = xp0*mod
    mask = n_mon > 0
    ed['xp_mean'] = xp0[mask].mean()
    ed['xp_adj'] = xp1[mask].mean()
    ed['mod_ratio'] = mod[mask].mean()

for ed in enc_diffs:
    ed['xp_ratio'] = ed['xp']/160
    ed['xp_ratio_adj'] = ed['xp_adj']/enc_diffs[-1]['xp_adj']
    print('{:}: XP = {:.0f}; mod ratio = {:.2f}; adj ratio = {:.2f}; fin ratio = {:.2f}'.format(
        ed['diff'], ed['xp'], 
        ed['mod_ratio'], ed['mod_ratio']/enc_diffs[-1]['mod_ratio'],
       ed['xp']*ed['mod_ratio']/(enc_diffs[-1]['mod_ratio']*enc_diffs[-1]['xp'])))
    #print('{:}: XP = {:.0f}; XP mean = {:.0f}; XP adj = {:.0f}; XP ratio = {:.2f}; adj XP ratio = {:.2f}'.format(ed['diff'], ed['xp'], ed['xp_mean'], ed['xp_adj'], ed['xp_ratio'], ed['xp_ratio_adj']))


Trivial: XP = 40; mod ratio = 0.50; adj ratio = 0.55; fin ratio = 0.14
Low: XP = 60; mod ratio = 0.61; adj ratio = 0.67; fin ratio = 0.25
Moderate: XP = 80; mod ratio = 0.67; adj ratio = 0.74; fin ratio = 0.37
Severe: XP = 120; mod ratio = 0.82; adj ratio = 0.89; fin ratio = 0.67
Extreme: XP = 160; mod ratio = 0.92; adj ratio = 1.00; fin ratio = 1.00
